## Using the BISON API
The USGS provides an API for accessing species observation data. https://bison.usgs.gov/doc/api.jsp

This API is much better documented than the NWIS API, and we'll use it to dig a bit deeper into how the `requests` package can faciliate data access via APIs. 

* We'll begin by replicating the example API call they show on their web page:<br> 
[https://bison.usgs.gov/api/search.json?species=Bison bison&type=scientific_name&start=0&count=1](
https://bison.usgs.gov/api/search.json?species=Bison%20bison&type=scientific_name&start=0&count=1)

In [1]:
#First, import the wonderful requests module
import requests

* Now, we'll deconstruct the example URL into the service URL and parameters, saving the paramters as a dictionary. Note we are just providing a few of the parameters available through the [API](https://bison.usgs.gov/doc/api.jsp#opensearch). We could add more search criteria if we wanted, but for now we just want to grab the first 500 Bison records. 

In [2]:
# Construct the service URL as two components: the service URL and the request parameters
url = 'http://bison.usgs.gov/api/search.json'
params = {'species':'Bison bison',
          'type':'scientific_name',
          'start':'0',
          'count':'500'
         }

* With the components set as variables, we use the `requests.get()` function to send our request off to the server at the address provided, storing the servers response as a variable called `response`. 

In [3]:
#Send the request to the server and store the response as a variable
response = requests.get(url,params)

* This response object contains a number of properties and methods. Let's have a look at the reponse in raw text format. 

In [4]:
#View the reponse in text format
print(response.text)

{"occurrences":{"legend":{"fossil":350,"observation":822,"centroid":1,"specimen":1038,"unknown":6}},"total":2216,"searchTime":214,"offset":0,"data":[{"geo":"Yes","provider":"iNaturalist.org","name":"Bison bison","decimalLongitude":"-110.58052062988281","decimalLatitude":"44.65469741821289","occurrenceID":"1584423649","common_name":"American Bison, bison, American bison, Bisonte americano","basis":"Observation"},{"geo":"No","provider":"National Museum of Nature and Science, Japan","name":"Bison bison","occurrenceID":"234483078","common_name":"American Bison, bison, American bison, Bisonte americano","basis":"Specimen"},{"geo":"No","provider":"Charles R. Conner Museum","name":"Bison bison","occurrenceID":"673101210","common_name":"American Bison, bison, American bison, Bisonte americano","basis":"Specimen"},{"geo":"Yes","provider":"iNaturalist.org","name":"Bison bison","decimalLongitude":"-118.53312683105469","decimalLatitude":"33.543487548828125","occurrenceID":"1305146396","common_name

**Yikes**, that's much less readable than the NWIS output!

Well, that's because the response from the BISON server is in **JSON** format. JSON, short for *JavaScript Object Notation*, is a text document that stores information in `key`:`value` pairs, *much like a Python dictionary*. Still, it's a raw text object, but one that we convert into a Python dictionary using `requests`'s `json()` function to convert the servers response into a Python dictionary.

In [5]:
#Convert the response 
data = response.json()
type(data)

dict

* Ok, if it's a dictionary, what are it's keys? 

In [6]:
#List the keys in the returned JSON object
data.keys()

dict_keys(['occurrences', 'total', 'searchTime', 'offset', 'data', 'species', 'eezs', 'itemsPerPage', 'counties', 'type', 'georeferenced', 'states'])

* What are the values linked with the 'data' key?

In [7]:
#Show the value associated with the `data` key
data['data']

[{'geo': 'Yes',
  'provider': 'iNaturalist.org',
  'name': 'Bison bison',
  'decimalLongitude': '-110.58052062988281',
  'decimalLatitude': '44.65469741821289',
  'occurrenceID': '1584423649',
  'common_name': 'American Bison, bison, American bison, Bisonte americano',
  'basis': 'Observation'},
 {'geo': 'No',
  'provider': 'National Museum of Nature and Science, Japan',
  'name': 'Bison bison',
  'occurrenceID': '234483078',
  'common_name': 'American Bison, bison, American bison, Bisonte americano',
  'basis': 'Specimen'},
 {'geo': 'No',
  'provider': 'Charles R. Conner Museum',
  'name': 'Bison bison',
  'occurrenceID': '673101210',
  'common_name': 'American Bison, bison, American bison, Bisonte americano',
  'basis': 'Specimen'},
 {'geo': 'Yes',
  'provider': 'iNaturalist.org',
  'name': 'Bison bison',
  'decimalLongitude': '-118.53312683105469',
  'decimalLatitude': '33.543487548828125',
  'occurrenceID': '1305146396',
  'common_name': 'American Bison, bison, American bison, Biso

* Oh, it's a list of occurrences! Let's examine the first one...

In [8]:
#Display the first "data" value
data['data'][0]

{'geo': 'Yes',
 'provider': 'iNaturalist.org',
 'name': 'Bison bison',
 'decimalLongitude': '-110.58052062988281',
 'decimalLatitude': '44.65469741821289',
 'occurrenceID': '1584423649',
 'common_name': 'American Bison, bison, American bison, Bisonte americano',
 'basis': 'Observation'}

* We see it's a dictionary too! Let's list the `decimalLatitude` item value...

In [9]:
#We can get the latitude of the record from it's `decimalLatitude` key
data['data'][0]['decimalLatitude']

'44.65469741821289'

► **So** we see the Bison observations are stored as list of dictionaries which are accessed within the `data` key in the results dictionary generated from the JSON response to our API request. (Phew!)

* With a bit more code we can loop through all the data records and print out the lat and long coordinates...

In [10]:
#Loop thorough each observation and print the lat and long values
for observation in data['data']:
    print (observation['decimalLatitude'],observation['decimalLongitude'])

44.65469741821289 -110.58052062988281


KeyError: 'decimalLatitude'

<details>
    <summary>
► If the above throws an error, can you debug it? HINT: the `geo` tag indicates whether coordinate info exist for the record...
    </summary>
    <pre><code>
#Loop thorough each observation and print the lat and long values
for observation in data['data']:
    if(observation['geo'] == 'Yes'):
        print (observation['decimalLatitude'],observation['decimalLongitude'])
    </code></pre>
</details>

In [11]:
#Loop thorough each observation and print the lat and long values
for observation in data['data']:
    if(observation['geo'] == 'Yes'):
        print (observation['decimalLatitude'],observation['decimalLongitude'])

44.65469741821289 -110.58052062988281
33.543487548828125 -118.53312683105469
33.212554931640625 -118.34165954589844
44.76667022705078 -110.23332977294922
44.425601959228516 -110.40275573730469
43.907325744628906 -102.26626586914062
44.57164764404297 -110.61011505126953
44.47914123535156 -110.49961853027344
33.47019958496094 -118.40839385986328
44.46189498901367 -110.83560180664062
44.571075439453125 -110.44855499267578
44.7253303527832 -110.57919311523438
43.763973236083984 -103.4005126953125
44.79423904418945 -110.43981170654297
44.554386138916016 -110.35734558105469
41.030860900878906 -112.24024200439453
36.282188415527344 -112.0276870727539
44.67873001098633 -110.53326416015625
48.84968566894531 -125.3843765258789
48.84968566894531 -125.3843765258789
33.68899917602539 -101.99800109863281
43.31148147583008 -118.79496765136719
44.43022918701172 -110.85151672363281
36.663490295410156 -96.109375
37.93645095825195 -110.87442016601562
34.754825592041016 -98.68489074707031
43.054443359375 

### Using Pandas to streamline the process...
Pandas can create a dataframe directly from dictionary values. 

In [12]:
import pandas as pd
df = pd.DataFrame(data['data'])
df.head()

,geo,provider,name,decimalLongitude,decimalLatitude,occurrenceID,common_name,basis
0,Yes,iNaturalist.org,Bison bison,-110.58052062988281,44.65469741821289,1584423649,"American Bison, bison, American bison, Bisonte...",Observation
1,No,"National Museum of Nature and Science, Japan",Bison bison,NaN,NaN,234483078,"American Bison, bison, American bison, Bisonte...",Specimen
2,No,Charles R. Conner Museum,Bison bison,NaN,NaN,673101210,"American Bison, bison, American bison, Bisonte...",Specimen
3,Yes,iNaturalist.org,Bison bison,-118.53312683105469,33.543487548828125,1305146396,"American Bison, bison, American bison, Bisonte...",Observation
4,Yes,iNaturalist.org,Bison bison,-118.34165954589844,33.212554931640625,1806324445,"American Bison, bison, American bison, Bisonte...",Observation


So now we can use our Panda's know-how to do some nifty analyses, including subsetting records for a specific provider.
* First we'll get a list of unique providers found in the data

In [13]:
#Generate a list of providers
df.provider.unique()

array(['iNaturalist.org', 'National Museum of Nature and Science, Japan',
       'Charles R. Conner Museum',
       'National Museum of Natural History, Smithsonian Institution',
       'University of Wyoming Museum of Vertebrates',
       'University of Alaska Museum of the North',
       'Canadian Museum of Nature',
       'University of Arkansas Collections Facility, UAFMC',
       'Museum of Texas Tech University (TTU)',
       'James R. Slater Museum of Natural History',
       'Gothenburg Natural History Museum (GNM)',
       'Sam Noble Oklahoma Museum of Natural History',
       'California Academy of Sciences',
       'Santa Barbara Museum of Natural History',
       'Fort Hays Sternberg Museum of Natural History',
       'University of Kansas Biodiversity Institute',
       'European Molecular Biology Laboratory (EMBL)',
       'Vanderbilt University',
       'Texas A&M University Biodiversity Research and Teaching Collections',
       'New Mexico Museum of Natural History and

* Now, we'll subset the rows that include that provider...

In [14]:
df.query("provider == 'iNaturalist.org'")

,geo,provider,name,decimalLongitude,decimalLatitude,occurrenceID,common_name,basis
0,Yes,iNaturalist.org,Bison bison,-110.58052062988281,44.65469741821289,1584423649,"American Bison, bison, American bison, Bisonte...",Observation
3,Yes,iNaturalist.org,Bison bison,-118.53312683105469,33.543487548828125,1305146396,"American Bison, bison, American bison, Bisonte...",Observation
4,Yes,iNaturalist.org,Bison bison,-118.34165954589844,33.212554931640625,1806324445,"American Bison, bison, American bison, Bisonte...",Observation
7,Yes,iNaturalist.org,Bison bison,-110.40275573730469,44.425601959228516,1621804816,"American Bison, bison, American bison, Bisonte...",Observation
8,Yes,iNaturalist.org,Bison bison,-102.26626586914062,43.907325744628906,1838041919,"American Bison, bison, American bison, Bisonte...",Observation
...,...,...,...,...,...,...,...,...
491,Yes,iNaturalist.org,Bison bison,-110.97702026367188,44.455997467041016,1571122071,"American Bison, bison, American bison, Bisonte...",Observation
493,Yes,iNaturalist.org,Bison bison,-110.64454650878906,44.522247314453125,891092440,"American Bison, bison, American bison, Bisonte...",Observation
495,Yes,iNaturalist.org,Bison bison,-110.24312591552734,44.90949630737305,891103292,"American Bison, bison, American bison, Bisonte...",Observation
497,Yes,iNaturalist.org,Bison bison,-110.45496368408203,44.85490036010742,891154235,"American Bison, bison, American bison, Bisonte...",Observation


In [15]:
df.dtypes

geo                 object
provider            object
name                object
decimalLongitude    object
decimalLatitude     object
occurrenceID        object
common_name         object
basis               object
dtype: object

## Exercise:
* Extract the first 500 red wolf (*"Canis rufus"*) records from the BISON API. 
* Can you create a table listing the records collected by the `University of Kansas Biodiversity Institute`?
* *Challenge*: Can you create a table listing all the records collected in North Carolina?

In [16]:
# Construct the service URL as two components: the service URL and the request parameters
url = 'http://bison.usgs.gov/api/search.json'
params = {'species':'Canis rufus',
          'type':'scientific_name',
          'start':'0',
          'count':'500'
         }

In [19]:
#Send the request to the server and store the response as a variable
response = requests.get(url,params)

In [20]:
#Convert the response 
data = response.json()
type(data)

dict

In [21]:
df = pd.DataFrame(data['data'])
df.head()

,geo,provider,name,occurrenceID,common_name,basis,decimalLongitude,decimalLatitude
0,No,"National Museum of Natural History, Smithsonia...",Canis rufus,1321421569,,Specimen,NaN,NaN
1,No,Carnegie Museums,Canis rufus,1271935147,,Fossil,NaN,NaN
2,No,"National Museum of Natural History, Smithsonia...",Canis rufus,1319244113,,Specimen,NaN,NaN
3,No,"National Museum of Natural History, Smithsonia...",Canis rufus,1320034479,,Specimen,NaN,NaN
4,No,"National Museum of Natural History, Smithsonia...",Canis rufus,1320051840,,Specimen,NaN,NaN


In [23]:
#Generate a list of providers
df.provider.unique()

array(['National Museum of Natural History, Smithsonian Institution',
       'Carnegie Museums', 'Museum of Vertebrate Zoology',
       'University of Washington Burke Museum',
       'Museum of Comparative Zoology, Harvard University',
       'James R. Slater Museum of Natural History',
       'Museum of Southwestern Biology',
       'Louisiana State University Museum of Natural Science', 'BISON',
       'University of Michigan Museum of Zoology',
       'Florida Museum of Natural History',
       'North Carolina State Museum of Natural Sciences', 'NatureServe'],
      dtype=object)

In [24]:
df.query("provider == 'North Carolina State Museum of Natural Sciences'")

,geo,provider,name,occurrenceID,common_name,basis,decimalLongitude,decimalLatitude
479,Yes,North Carolina State Museum of Natural Sciences,Canis rufus,897160742,,Specimen,-75.79190063476562,35.60390090942383
480,Yes,North Carolina State Museum of Natural Sciences,Canis rufus,897160743,,Specimen,-75.79190063476562,35.60390090942383
481,Yes,North Carolina State Museum of Natural Sciences,Canis rufus,897160744,,Specimen,-75.79190063476562,35.60390090942383
482,Yes,North Carolina State Museum of Natural Sciences,Canis rufus,897160745,,Specimen,-75.79190063476562,35.60390090942383
483,Yes,North Carolina State Museum of Natural Sciences,Canis rufus,897166869,,Specimen,-75.79190063476562,35.60390090942383
484,Yes,North Carolina State Museum of Natural Sciences,Canis rufus,897166870,,Specimen,-75.79190063476562,35.60390090942383
